In [1]:
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

%env TMPDIR=/tmp/colima

env: TMPDIR=/tmp/colima


In [3]:
import re


df = pd.read_parquet("upgrades/research_data.parquet")
df

,url,project,projectOrganisation,breakingCommit,prAuthor,preCommitAuthor,breakingCommitAuthor,preCommitReproductionCommand,breakingUpdateReproductionCommand,javaVersionUsedForReproduction,...,updatedDependency.dependencySection,updatedDependency.licenseInfo,updatedDependency.githubRepoSlug,japicmpResult,elementLines,elementPatterns,allPotentialBreakingElements,revapiResult,failureType,rawFailures
2,https://github.com/OCR4all/OCR4all/pull/149,OCR4all,OCR4all,bd3ce213e2771c6ef7817c80818807a757d4e94a,bot,human,bot,docker run ghcr.io/chains-project/breaking-upd...,docker run ghcr.io/chains-project/breaking-upd...,11,...,dependencies,Apache-2.0,fasterxml/jackson-databind,"{'AlertDescription': None, 'BasicRedisPipeline...","{'Builder': None, 'ListenableFutureCallback': ...","{'Builder': None, 'ListenableFutureCallback': ...","[writeValue(java.io.File,java.lang.Object)]","{'Builder': None, 'ListenableFutureCallback': ...",DEPENDENCY_BREAKING_API_CHANGE,
5,https://github.com/feedzai/pdb/pull/365,pdb,feedzai,43b3a858b77ec27fc8946aba292001c3de465012,bot,bot,bot,docker run ghcr.io/chains-project/breaking-upd...,docker run ghcr.io/chains-project/breaking-upd...,11,...,dependencies,NOASSERTION,qos-ch/logback,"{'AlertDescription': None, 'BasicRedisPipeline...","{'Builder': None, 'ListenableFutureCallback': ...","{'Builder': None, 'ListenableFutureCallback': ...","[setLevel(ch.qos.logback.classic.Level), getLo...","{'Builder': None, 'ListenableFutureCallback': ...",DEPENDENCY_BREAKING_API_CHANGE,
6,https://github.com/jadler-mocking/jadler/pull/207,jadler,jadler-mocking,d3af06df4613be146bb9f8034e1a8a3098050c82,bot,human,bot,docker run ghcr.io/chains-project/breaking-upd...,docker run ghcr.io/chains-project/breaking-upd...,11,...,dependencyManagement,NOASSERTION,jetty/jetty.project,"{'AlertDescription': None, 'BasicRedisPipeline...","{'Builder': None, 'ListenableFutureCallback': ...","{'Builder': None, 'ListenableFutureCallback': ...",[org.eclipse.jetty.server.nio.SelectChannelCon...,"{'Builder': None, 'ListenableFutureCallback': ...",DEPENDENCY_BREAKING_API_CHANGE,
7,https://github.com/googleapis/java-datastore/p...,java-datastore,googleapis,4d3c6dbc0bd9ca66b24af0bafda70c7355c3daf7,bot,bot,bot,docker run ghcr.io/chains-project/breaking-upd...,docker run ghcr.io/chains-project/breaking-upd...,11,...,dependencyManagement,None,None,None,None,None,None,None,DEPENDENCY_VERSION_MISMATCH,
8,https://github.com/jadler-mocking/jadler/pull/160,jadler,jadler-mocking,ae16b526695fe275ab5e6a1992916875d26da860,bot,human,bot,docker run ghcr.io/chains-project/breaking-upd...,docker run ghcr.io/chains-project/breaking-upd...,11,...,dependencyManagement,NOASSERTION,jetty/jetty.project,"{'AlertDescription': None, 'BasicRedisPipeline...","{'Builder': None, 'ListenableFutureCallback': ...","{'Builder': None, 'ListenableFutureCallback': ...",[org.eclipse.jetty.server.nio.SelectChannelCon...,"{'Builder': None, 'ListenableFutureCallback': ...",DEPENDENCY_BREAKING_API_CHANGE,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,https://github.com/googleapis/java-pubsub-grou...,java-pubsub-group-kafka-connector,googleapis,78252d13ff355a0205fff6fb7c19791412bc6478,bot,bot,bot,docker run ghcr.io/chains-project/breaking-upd...,docker run ghcr.io/chains-project/breaking-upd...,11,...,dependencyManagement,Apache-2.0,googleapis/java-pubsublite,None,None,None,None,None,DEPENDENCY_VERSION_MISMATCH,
564,https://github.com/googleapis/java-bigquery/pu...,java-bigquery,googleapis,e05121a99cab7dc2b7c88094c2aa2a43e453ee68,bot,bot,bot,docker run ghcr.io/chains-project/breaking-upd...,docker run ghcr.io/chains-project/breaking-upd...,11,...,dependencyManagement,Apache-2.0,apache/arrow,None,None,None,None,None,DEPENDENCY_VERSION_MISMATCH,
566,https://github.com/googleapis/java-translate/p...,java-translate,googleapis,5e61a76ca214c41d7e1c079e6a81c3ba9c26772e,bot,bot,bot,docker run ghcr.io/chains-project/breaking-upd...,docker run ghcr.io/chains-proje

In [4]:
import json
from pathlib import Path

revapi_path = Path("bump/RQData/japicmp-revapi-analysis-results.json")

with revapi_path.open("r", encoding="utf-8") as f:
    revapi_dict = json.load(f)

# sample 5 keys
sampled_keys = [
    revapi_item
    for revapi_item in revapi_dict.keys()
    if revapi_dict[revapi_item]["japicmpResult"] != {}
    or revapi_dict[revapi_item]["revapiResult"] != {}
    or revapi_dict[revapi_item]["elementLines"] != {}
]
sampled_keys = sampled_keys[20:40]
candidates = sampled_keys

log_regex = r"(WARNING|ERROR)\] (.+\.java):\["


def get_files_with_errors(element_lines):
    files = []
    for line in element_lines:
        match = re.search(log_regex, line)
        if match:
            path = match.group(2)
            # split off the first folder of the path
            path = path.split("/")[2:]
            files.append("/".join(path))
    return files

We want a dynamic interface to cleanly apply diffs to the tmpdir, create a new Dockerfile that incoporates the changes and build the image
We want to be able to run language server on the project 
For now we want to focus on one handpicked class that should fix an easy problem

In [5]:
from dspy import InputField, OutputField, Signature, settings, Example
from dspy.functional import TypedPredictor
from dspy import OpenAI
import dspy
from dotenv import load_dotenv

from enum import Enum

load_dotenv(".env")  # load OpenAI API key from .env file
# gpt35 = OpenAI(model="gpt-3.5-turbo", max_tokens=4000)
gpt35 = OpenAI(model="gpt-4o", max_tokens=4096)

# litellm
# lm = OpenAI(model="gpt-3.5-turbo",
#     api_key="anything",
#     api_base="http://0.0.0.0:8000")

# anthropic = OpenAI(model="claude-3-5-sonnet-20240620",
#     api_key="anything",
#     api_base="http://0.0.0.0:8000")
settings.configure(lm=gpt35)


# assert that /var/run/docker.sock exists
assert Path("/var/run/docker.sock").exists()

/Users/anon/Projects/masterthesis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from collections import defaultdict
import subprocess
import tempfile
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
from dspy.predict import Retry
from masterthesis.agent.DockerAgent import DockerAgent
from masterthesis.agent.LSPAgent import LSPAgent
from masterthesis.agent.MarkdownAgent import MarkdownAgent
from masterthesis.agent.DiffAgent import DiffAgent
import os

from masterthesis.agent.aider.AdvancedDiffAgent import UnifiedDiffCoder
from masterthesis.llm.pipeline import pipeline

os.environ["TMPDIR"] = "/tmp/colima"

import tempfile

tempfile.tempdir = "/tmp/colima"

# import tiktoken
# enc = tiktoken.encoding_for_model("gpt-4o")

candidate_results = defaultdict(dict)


# Inspired by https://github.com/paul-gauthier/aider/blob/main/aider/coders/udiff_prompts.py

system_prompt = """Act as an expert software developer
Always use best practices when coding.
Respect and use existing conventions, libraries, etc that are already present in the code base.
Adapt the program to the changes in the dependency API that happened during a version upgrade of the dependency

# File editing rules:

Return edits similar to unified diffs that `diff -U0` would produce.

Make sure you include the first 2 lines with the file paths.
Don't include timestamps with the file paths.

Start each hunk of changes with a `@@ ... @@` line.
Don't include line numbers like `diff -U0` does.
The user's patch tool doesn't need them.

The user's patch tool needs CORRECT patches that apply cleanly against the current contents of the file!
Think carefully and make sure you include and mark all lines that need to be removed or changed as `-` lines.
Make sure you mark all new or modified lines with `+`.
Don't leave out any lines or the diff patch won't apply correctly.

Indentation matters in the diffs!

Start a new hunk for each section of the file that needs changes.

Only output hunks that specify changes with `+` or `-` lines.
Skip any hunks that are entirely unchanging ` ` lines.

Output hunks in whatever order makes the most sense.
Hunks don't need to be in any particular order.

When editing a function, method, loop, etc use a hunk to replace the *entire* code block.
Delete the entire existing version with `-` lines and then add a new, updated version with `+` lines.
This will help you generate correct code and correct diffs.

To move code within a file, use 2 hunks: 1 to delete it from its current location, 1 to insert it in the new location.
"""

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_prompt),
        ("human", "{query}"),
    ]
).partial(schema=People.schema())


# Todo: In the loop this might not be tooo clever
candidate_results[commit_hash]["hop"] = hop


def invalid_diff_callback(is_valid_diff, diff_remarks):
    dspy.Assert(is_valid_diff, f"Invalid diff: {diff_remarks}")


def diagnostic_callback(diagnostics):
    # print("Initial diagnostics", flush=True)
    # print("-"*20, flush=True)

    # print(diagnostics, flush=True)

    # print("-"*20, flush=True)

    dspy.Suggest(len(diagnostics) > 0, f"Linting failed: {diagnostics}")
    if len(diagnostics) == 0:
        print("Linting passed", flush=True)
    else:
        print("Suggested", diagnostics, flush=True)


def compile_callback(has_errors, error_text):
    dspy.Assert(not has_errors, f"Error during compilation/test: {error_text}")
    if has_errors:
        print(f"Error during compilation: {error_text}", flush=True)
    else:
        print("Compilation passed", flush=True)


def generate_diff_callback(
    code, path, api_changes, initial_error, updated_dependency_diff
):
    candidate_results[commit_hash]["input"]["prepared_file"] = code

    extracted_diff = self.generate_diff(
        code=code,
        path=path,
        api_changes=api_changes,
        initial_error=initial_error,
        updated_dependency_diff=updated_dependency_diff,
    ).answer
    candidate_results[commit_hash]["raw_output"] = extracted_diff
    return extracted_diff


print(f'OpenAI: {openai_response.response_metadata["token_usage"]}\n')

return pipeline(
    param_token_count,
    file_in_scope,
    initial_error_lines,
    updated_dependency_diff,
    api_changes,
    repo_slug,
    commit_hash,
    reproduction_command,
    invalid_diff_callback,
    diagnostic_callback,
    compile_callback,
    generate_diff_callback,
)


fix_code_with_assertions = assert_transform_module(
    FixCode().map_named_predictors(Retry), backtrack_handler
)

In [7]:
import traceback
import requests

from masterthesis.agent.LSPAgent import extract_error_lines

import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

for commit_hash in candidates:
    print(f"Processing {commit_hash}")
    test_candidate = df[df["breakingCommit"] == commit_hash]

    if len(test_candidate) == 0:
        print(f"No test candidate found for {commit_hash}")
        continue

    repository_name = test_candidate["project"].values[0]
    repo_slug = test_candidate["projectOrganisation"].values[0] + "/" + repository_name

    revapi = revapi_dict.get(
        commit_hash, {"elementLines": {}, "allPotentialBreakingElements": {}}
    )
    suspicious_files = list(set(get_files_with_errors(revapi["elementLines"].values())))
    if len(suspicious_files) == 0:
        print(f"No revapi log found for {commit_hash}")
        continue

    api_changes = json.dumps(revapi["allPotentialBreakingElements"])
    file_in_scope = suspicious_files[0]

    reproduction_log_path = Path("bump/reproductionLogs/successfulReproductionLogs") / (
        commit_hash + ".log"
    )
    if not reproduction_log_path.exists():
        raise ValueError(f"Reproduction log for {commit_hash} does not exist")
    with open(reproduction_log_path, "r") as f:
        reproduction_log = f.read()
        initial_error_lines = "\n".join(extract_error_lines(reproduction_log))

    response = requests.get(test_candidate["url"].values[0] + ".diff")

    assert response.status_code == 200, f"Could not download diff for {commit_hash}"
    updated_dependency_diff = response.text

    input_params = {
        "initial_error_lines": initial_error_lines,
        "api_changes": api_changes,
        "repo_slug": repo_slug,
        "commit_hash": commit_hash,
        "reproduction_command": "mvn clean test -B -Dsurefire.printSummary=true -Dsurefire.redirectTestOutputToFile=false",
        "updated_dependency_diff": updated_dependency_diff,
    }

    input_params["file_in_scope"] = file_in_scope
    input_params["param_token_count"] = sum(
        [len(enc.encode(input_params[k])) for k in input_params.keys()]
    )

    candidate_results[commit_hash] = {
        "patch": None,
        "error": None,
        "input": input_params,
    }

    try:
        candidate_results[commit_hash]["output"] = fix_code_with_assertions(
            **input_params
        )
    except Exception as e:
        print(traceback.format_exc())
        print(f"Error processing {commit_hash}: {e}")
        candidate_results[commit_hash]["error"] = str(e)

with open("errors.json", "w", encoding="utf-8") as f:
    json.dump(candidate_results, f, ensure_ascii=True, indent=4)

Processing 874ed893a4e46ea5182be2be054715967e58f08f
Pulling image ghcr.io/lukvonstrom/multilspy-java-docker:latest...
Pulling image ghcr.io/lukvonstrom/multilspy-java-docker:latest...
Started container
Executing command /bin/bash -c '/app/init.sh -e yes -f src/main/java/de/gwdg/metadataqa/marc/dao/MarcRecord.java'
Suggested ['[JAVA] 427:13 to 427:29 - JacksonException cannot be resolved to a type', '[JAVA] 427:13 to 427:29 - JacksonException cannot be resolved to a type', '[JAVA] 427:13 to 427:29 - JacksonException cannot be resolved to a type', '[JAVA] 427:13 to 427:29 - JacksonException cannot be resolved to a type']
No context lines provided, no levensthein match possible
Pulling image ghcr.io/lukvonstrom/multilspy-java-docker:latest...
Pulling image ghcr.io/lukvonstrom/multilspy-java-docker:latest...
Started container
No context lines provided, no levensthein match possible
Executing command /bin/bash -c '/app/init.sh -e yes -f src/main/java/de/gwdg/metadataqa/marc/dao/MarcRecord.j

In [8]:
examples = gpt35.inspect_history(n=10)





Act as an expert software developer
Always use best practices when coding.
Respect and use existing conventions, libraries, etc that are already present in the code base.
Adapt the program to the changes in the dependency API that happened during a version upgrade of the dependency

# File editing rules:

Return edits similar to unified diffs that `diff -U0` would produce.

Make sure you include the first 2 lines with the file paths.
Don't include timestamps with the file paths.

Start each hunk of changes with a `@@ ... @@` line.
Don't include line numbers like `diff -U0` does.
The user's patch tool doesn't need them.

The user's patch tool needs CORRECT patches that apply cleanly against the current contents of the file!
Think carefully and make sure you include and mark all lines that need to be removed or changed as `-` lines.
Make sure you mark all new or modified lines with `+`.
Don't leave out any lines or the diff patch won't apply correctly.

Indentation matters in the dif